# Building Supply Optimization

Optimal combination of segments and periods for building energy supply systems.

Author: Leander Kotzur

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd

# Configure Plotly for sphinx/nbsphinx output
import plotly.io as pio

import tsam
from tsam import ClusterConfig, SegmentConfig
from tsam.tuning import find_pareto_front

pio.renderers.default = "notebook"

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)
raw = raw.rename(
    columns={
        "T": "Temperature [°C]",
        "Load": "Load [kW]",
        "Wind": "Wind [m/s]",
        "GHI": "Solar [W/m²]",
    }
)

In [ ]:
raw.drop(
    columns=[
        "Wind [m/s]",
    ],
    inplace=True,
)

Use tsam's built-in heatmap plotting for visual comparison of the time series

Plot an example series - in this case the temperature

In [ ]:
# Original data heatmaps
tsam.plot.heatmaps(raw, period_hours=24, title="Original Data")

### Tune a hierarchical aggregation with segments in combination with duration representation

Use the new `find_pareto_front()` function to explore the Pareto-optimal combinations.

In [ ]:
pareto_results = find_pareto_front(
    raw,
    period_hours=24,
    max_timesteps=100,
    cluster=ClusterConfig(
        method="hierarchical",
        representation="duration",
    ),
)

And determine the pareto optimal aggregation up to 200 total time steps. This may take some time...

In [ ]:
# Show the last result in the Pareto front
last_result = pareto_results[-1]
print(f"Final configuration: {last_result.optimal_n_periods} periods, {last_result.optimal_n_segments} segments")

And show the results for the last aggregation

In [ ]:
# Reconstruct the data from the last Pareto result
reconstructed = last_result.best_result.reconstruct()

In [ ]:
# Reconstructed data heatmaps from last tuned aggregation
tsam.plot.heatmaps(
    reconstructed,
    reference_data=raw,
    period_hours=24,
    title="Reconstructed Data (Tuned)",
)

In [ ]:
last_result.optimal_n_segments

In [ ]:
last_result.optimal_n_periods

In [ ]:
# Example with specific configuration using distribution_minmax representation
result = tsam.aggregate(
    raw,
    n_periods=14,
    period_hours=24,
    cluster=ClusterConfig(
        method="hierarchical",
        representation="distribution_minmax",
    ),
    segments=SegmentConfig(n_segments=8),
    rescale=False,
)

In [ ]:
# Reconstructed data heatmaps with 8 segments and 14 periods
tsam.plot.heatmaps(
    result.reconstruct(),
    reference_data=raw,
    period_hours=24,
    title="Reconstructed Data (8 segments, 14 periods)",
)